In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

In [2]:
input = np.random.randint(2, size=(1000, 5))
output = np.concatenate((input, input), axis=1)
noise = np.random.choice([0, 1], size=output.shape, p=[0.9, 0.1])
output = output ^ noise

In [3]:
cls = MultiOutputClassifier(LogisticRegression()).fit(input, output)

In [4]:
cls.predict_proba(input[:1])

[array([[0.12623232, 0.87376768]]),
 array([[0.06630806, 0.93369194]]),
 array([[0.9080659, 0.0919341]]),
 array([[0.8803219, 0.1196781]]),
 array([[0.88283731, 0.11716269]]),
 array([[0.10962904, 0.89037096]]),
 array([[0.08200295, 0.91799705]]),
 array([[0.79956396, 0.20043604]]),
 array([[0.87107395, 0.12892605]]),
 array([[0.86754249, 0.13245751]])]

In [5]:
input[:1]

array([[1, 1, 0, 0, 0]])

In [6]:
import torch
from torch.nn import Module, Linear, BCEWithLogitsLoss
from torch.optim import SGD
from torch import special

In [7]:
class Dumpy(Module):
    def __init__(self):
        super(Dumpy, self).__init__()
        self.linear = Linear(in_features=5, out_features=10, bias=True)
    
    def forward(self, x):
        x = self.linear(x)
        return special.expit(x)

    def forward_logit(self, x):
        return self.linear(x)

In [8]:
x = torch.tensor(input, dtype=torch.float32)
y = torch.tensor(output, dtype=torch.float32)

cls_pt = Dumpy()
optimizer = SGD(cls_pt.parameters(), lr=1)
bcewl = BCEWithLogitsLoss()

In [9]:
EPOCHS = 10_000

for epoch in range(EPOCHS):
    optimizer.zero_grad()
    model_output = cls_pt.forward_logit(x)
    loss = bcewl(model_output, y)
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss.item())

0.7359381318092346
0.319843590259552
0.3159427344799042
0.31549105048179626
0.31541693210601807
0.315403550863266
0.3154010772705078
0.3154005706310272
0.31540045142173767
0.31540045142173767


In [10]:
cls_pt.forward(x[:1])

tensor([[0.8829, 0.9432, 0.0820, 0.1117, 0.1079, 0.9001, 0.9282, 0.1967, 0.1213,
         0.1252]], grad_fn=<SigmoidBackward0>)